In [ ]:
# Importar librerías necesarias
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np
import os

# Solicitar el archivo txt al usuario
from google.colab import files

print("Por favor, sube el archivo .txt con las intervenciones del parlamentario.")
uploaded = files.upload()

# Leer el archivo .txt
filename = list(uploaded.keys())[0]
with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()

# Preprocesamiento del texto
def preprocess_text(text):
    text = text.lower()  # Convertir a minúsculas
    text = text.replace('\n', ' ')  # Reemplazar saltos de línea por espacios
    text = ''.join(c for c in text if c.isalnum() or c.isspace())  # Eliminar puntuación
    return text

text = preprocess_text(text)
words = text.split()  # Dividir el texto en palabras únicas
vocab = sorted(set(words))  # Crear vocabulario único

# Crear diccionarios de mapeo
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for word, i in word_to_idx.items()}

# Convertir el texto a índices
encoded_text = [word_to_idx[word] for word in words]

# Preparar los datos para el modelo
sequence_length = 10  # Longitud de las secuencias
X = []
y = []

for i in range(len(encoded_text) - sequence_length):
    X.append(encoded_text[i:i+sequence_length])
    y.append(encoded_text[i+sequence_length])

X = np.array(X)
y = np.array(y)

# Construir el modelo LSTM de 4 capas con dropout y optimización
vocab_size = len(vocab)
embedding_dim = 128  # Dimensión del embedding
lstm_units = 512  # Número de unidades en LSTM

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=sequence_length),
    LSTM(lstm_units, return_sequences=True),
    Dropout(0.4),  # Dropout del 40%
    LSTM(lstm_units, return_sequences=True),
    Dropout(0.4),
    LSTM(lstm_units, return_sequences=True),
    Dropout(0.4),
    LSTM(lstm_units),
    Dropout(0.4),
    Dense(vocab_size, activation='softmax')
])

# Compilar el modelo
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005), metrics=['accuracy'])

# Guardar el mejor modelo durante el entrenamiento
checkpoint = ModelCheckpoint('modelo_LSTM_parlamentario_parrafo.keras', monitor='loss', save_best_only=True)

# Entrenar el modelo con más épocas para mejor rendimiento
print("Entrenando el modelo durante más épocas...")
model.fit(X, y, epochs=200, batch_size=128, verbose=1, callbacks=[checkpoint])

# Función para generar texto como párrafo con múltiples frases
def generate_paragraph(model, start_sequence, max_words=200, temperature=1.0):
    input_sequence = [word_to_idx[word] for word in start_sequence.split() if word in word_to_idx]
    generated = start_sequence
    sentence_count = 0

    for _ in range(max_words):
        input_padded = tf.keras.preprocessing.sequence.pad_sequences([input_sequence], maxlen=sequence_length)
        predictions = model.predict(input_padded, verbose=0).flatten()
        predictions = np.log(predictions + 1e-9) / temperature
        probabilities = np.exp(predictions) / np.sum(np.exp(predictions))
        predicted_idx = np.random.choice(len(probabilities), p=probabilities)
        predicted_word = idx_to_word[predicted_idx]

        generated += ' ' + predicted_word
        input_sequence.append(predicted_idx)
        input_sequence = input_sequence[1:]

        # Contar oraciones según puntuación
        if predicted_word in ['.', '!', '?']:
            sentence_count += 1
            if sentence_count >= 5:  # Terminar tras 5 oraciones
                break

    return generated

# Interactuar con el modelo generando párrafos
print("¡Puedes comenzar a interactuar con el modelo!")
while True:
    prompt = input("Introduce el inicio del texto (o escribe 'salir' para terminar): ")
    if prompt.lower() == 'salir':
        print("¡Hasta luego!")
        break
    print("Generando párrafo...")
    print(generate_paragraph(model, prompt, max_words=200, temperature=0.8))


Por favor, sube el archivo .txt con las intervenciones del parlamentario.


Saving intervencionesCasado.txt to intervencionesCasado.txt


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Entrenando el modelo durante más épocas...
Epoch 1/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0407 - loss: 7.8766
Epoch 2/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0416 - loss: 6.2848
Epoch 3/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0437 - loss: 6.2944
Epoch 4/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0443 - loss: 6.2775
Epoch 5/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0444 - loss: 6.2737
Epoch 6/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0437 - loss: 6.2731
Epoch 7/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0433 - loss: 6.2691
Epoch 8/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0453 - loss: 6.2134
Epoch 9/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0480 - loss: 6.1969
Epoch 10/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0423 - loss: 6.1922
Epoch 11/200
143/143 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0444 -